# Homework

Dataset

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. For this, we will use the "Bee or Wasp?" dataset that was obtained from Kaggle and slightly rebuilt.

You can download the dataset for this homework from here:

In [1]:
! wget -O data.zip https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
! unzip -qq data.zip -d data

--2023-11-12 16:44:28--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
Resolving github.com (github.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘github.com’
[data.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data.zip or
        data.zip.zip, and cannot find data.zip.ZIP, period.


## Data Preparation

The dataset contains around 2500 images of bees and around 2100 images of wasps.

The dataset contains separate folders for training and test sets.

## Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

- The shape for input should be (150, 150, 3)
- Next, create a convolutional layer (Conv2D):
    - Use 32 filters
    - Kernel size should be (3, 3) (that's the size of the filter)
    - Use 'relu' as activation
- Reduce the size of the feature map with max pooling (MaxPooling2D)
    - Set the pooling size to (2, 2)
- Turn the multi-dimensional result into vectors using a Flatten layer
- Next, add a Dense layer with 64 neurons and 'relu' activation
- Finally, create the Dense layer with 1 neuron - this will be the output
    - The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:

- SGD(lr=0.002, momentum=0.8)

## Question 1

Since we have a binary classification problem, what is the best loss function for us?

- mean squared error
- binary crossentropy
- categorical crossentropy
- cosine similarity

In [2]:
# Install TensorFlow
#!conda install -c conda-forge tensorflow -y

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

2023-11-12 16:44:29.079019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# Building of Model
inputs = keras.Input(shape=(150,150,3))
x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
x = keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(units=64, activation='relu')(x)
outputs = keras.layers.Dense(units=1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [21]:
model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.SGD(learning_rate=.002, momentum=.8), 
    metrics=[keras.metrics.BinaryAccuracy()]
)

## Question 2

What's the number of parameters in the convolutional layer of our model? You can use the summary method for that.
- 1
- 65
- 896
- 11214912

In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 175232)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                11214912  
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                           

## Generators and Training

For the next two questions, use the following data generator for both train and test sets:
<code>
ImageDataGenerator(rescale=1./255)
</code>
- We don't need to do any additional pre-processing for the images.
- When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
- Use batch_size=20
- Use shuffle=True for both training and test sets.

For training use .fit() with the following params:
<code>
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
</code>

In [23]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [24]:
train_generator = train_datagen.flow_from_directory(directory='data/data/train',target_size=(150,150),class_mode='binary', batch_size=20)
test_generator = test_datagen.flow_from_directory(directory='data/data/test',target_size=(150,150),class_mode='binary', batch_size=20)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [25]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10


2023-11-12 22:52:36.413304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - ETA: 0s - loss: 0.6742 - binary_accuracy: 0.5706

2023-11-12 22:57:43.030154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - 328s 2s/step - loss: 0.6742 - binary_accuracy: 0.5706 - val_loss: 0.6297 - val_binary_accuracy: 0.6198
Epoch 2/10
184/184 [==============================] - 362s 2s/step - loss: 0.6367 - binary_accuracy: 0.6288 - val_loss: 0.5993 - val_binary_accuracy: 0.6906
Epoch 3/10
184/184 [==============================] - 314s 2s/step - loss: 0.5953 - binary_accuracy: 0.6927 - val_loss: 0.5639 - val_binary_accuracy: 0.7179
Epoch 4/10
184/184 [==============================] - 369s 2s/step - loss: 0.5502 - binary_accuracy: 0.7324 - val_loss: 0.5433 - val_binary_accuracy: 0.7418
Epoch 5/10
184/184 [==============================] - 324s 2s/step - loss: 0.5149 - binary_accuracy: 0.7561 - val_loss: 0.5346 - val_binary_accuracy: 0.7473
Epoch 6/10
184/184 [==============================] - 319s 2s/step - loss: 0.4995 - binary_accuracy: 0.7686 - val_loss: 0.5387 - val_binary_accuracy: 0.7386
Epoch 7/10
184/184 [==============================] - 207s 1s/step - 

In [26]:
history.history.keys()

dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

## Question 3

What is the median of training accuracy for all the epochs for this model?
- 0.20
- 0.40
- 0.60
- 0.80

In [27]:
train_accuracy_history = history.history['binary_accuracy']
np.median(train_accuracy_history).round(2)

0.76

## Question 4

What is the standard deviation of training loss for all the epochs for this model?
- 0.031
- 0.061
- 0.091
- 0.131

In [39]:
training_loss_history = history.history['loss']
((training_loss_history-np.mean(training_loss_history))**2).sum().round(4)

0.08

## Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:
- rotation_range=50,
- width_shift_range=0.1,
- height_shift_range=0.1,
- zoom_range=0.1,
- horizontal_flip=True,
- fill_mode='nearest'

In [29]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                                             rotation_range=50, 
                                                             width_shift_range=0.1, 
                                                             height_shift_range=0.1, 
                                                             zoom_range=0.1, 
                                                             horizontal_flip=True, 
                                                             fill_mode='nearest')
#test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(directory='data/data/train',target_size=(150,150),class_mode='binary', batch_size=20)
#test_generator = test_datagen.flow_from_directory(directory='data/data/test',target_size=(150,150),class_mode='binary', batch_size=20)

Found 3677 images belonging to 2 classes.


## Question 5

Let's train our model for 10 more epochs using the same code as previously.

    Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?
- 0.18
- 0.48
- 0.78
- 0.108

In [30]:
history_new = model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10


2023-11-12 23:42:22.860862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - ETA: 0s - loss: 0.4982 - binary_accuracy: 0.7653

2023-11-12 23:45:52.539441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - 220s 1s/step - loss: 0.4982 - binary_accuracy: 0.7653 - val_loss: 0.4841 - val_binary_accuracy: 0.7603
Epoch 2/10
184/184 [==============================] - 205s 1s/step - loss: 0.4855 - binary_accuracy: 0.7702 - val_loss: 0.4947 - val_binary_accuracy: 0.7614
Epoch 3/10
184/184 [==============================] - 205s 1s/step - loss: 0.4838 - binary_accuracy: 0.7775 - val_loss: 0.5138 - val_binary_accuracy: 0.7538
Epoch 4/10
184/184 [==============================] - 202s 1s/step - loss: 0.4779 - binary_accuracy: 0.7797 - val_loss: 0.4615 - val_binary_accuracy: 0.7876
Epoch 5/10
184/184 [==============================] - 313s 2s/step - loss: 0.4696 - binary_accuracy: 0.7792 - val_loss: 0.5106 - val_binary_accuracy: 0.7658
Epoch 6/10
184/184 [==============================] - 274s 1s/step - loss: 0.4672 - binary_accuracy: 0.7919 - val_loss: 0.4507 - val_binary_accuracy: 0.7832
Epoch 7/10
184/184 [==============================] - 235s 1s/step - 

In [31]:
evaluation = model.evaluate(test_generator)
print(f'Loss:{np.round(evaluation[0],3)}, binary_accuracy: {np.round(evaluation[1],3)}')

2023-11-13 00:20:00.433249: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


46/46 [==============================] - 11s 247ms/step - loss: 0.4411 - binary_accuracy: 0.7919
Loss:0.441, binary_accuracy: 0.792


## Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
- 0.38
- 0.58
- 0.78
- 0.98

In [32]:
np.mean(history_new.history['val_binary_accuracy'][-5:]).round(3)

0.785